<a href="https://colab.research.google.com/github/carlibeisel/Drains_Lower_Boise_River/blob/main/04_landcover_calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: March 14, 2022

Date modified: May 13, 2024

Purpose: Calculates annual percent for each land use from LCMAP data.

In [ ]:
## IMPORT PACKAGES ##

import numpy as np #basic computation
!pip install geopandas
import geopandas as gpd #geopandas for .shp
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time
import glob
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 230.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 907.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.6 MB/s eta 0:00:00


WARNING  Module /usr/local/lib/python3.10/dist-packages/pylandstats/landscape.py has not been compiled for         
         Transonic-Pythran

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
## ---------------------- ##
## SUBSET GEOSPATIAL DATA ##
## ---------------------- ##

shp_file = gpd.read_file('/content/gdrive/MyDrive/Data/drainage_analysis/drain_delineation/Drains_Merge_07072022.shp') #open shapefile
names = shp_file['Name']
files = glob.glob('/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/*.tiff') #get all the years of cdl imagery
data =[]
for i in range(len(files)):
  data.append(rso.open(files[i])) #open cdl image and append to a list
  print(rso.open(files[i]))
shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
years = np.arange(1987, 2021) #years of LCMAP data
collection = []
for i in range(len(shp)):
  for n in range(len(years)):
    dataset = data[n]
    year_out = dataset.name[64:68]
    print(year_out)
    extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
    coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
    out_img, out_transform = mask(dataset=data[n], shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
    out_meta = data[n].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform})
    # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
    in_file = files[n]
    out_filename = os.path.join('/content/gdrive/MyDrive/Data/drainage_analysis/land_use/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
    print(out_filename)
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
    out_tiff = rso.open(out_filename, 'w', **out_meta)
    out_tiff.write(np.squeeze(out_img, 0), 1)
    out_tiff.close()
    collection.append(out_img)

0                West Hartley
1              Sand Run Gulch
2                 Eagle Drain
3                 Mason Drain
4                 Mason Creek
5          East Hartley Drain
6                Willow Creek
7                Conway Gulch
8                ThurmanDrain
9             North Middleton
10            South Middleton
11               Indian Creek
12    Drainage District No. 3
13         Fifteen Mile Creek
14                Dixie drain
Name: Name, dtype: object
['/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1989_V12_LCPRI.tiff', '/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1986_V12_LCPRI.tiff', '/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1987_V12_LCPRI.tiff', '/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1988_V12_LCPRI.tiff', '/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1990_V12_LCPRI.tiff', '/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1992_V12_LCPRI.tiff', '

<open DatasetReader name='/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1989_V12_LCPRI.tiff' mode='r'>


<open DatasetReader name='/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1986_V12_LCPRI.tiff' mode='r'>


<open DatasetReader name='/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1987_V12_LCPRI.tiff' mode='r'>


<open DatasetReader name='/content/gdrive/MyDrive/Data/spatial_colab/lcmap_files/LCMAP_CU_1988_V12_LCPRI.tiff' mode='r'>


...

/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_2015.tif
2014
/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_2014.tif


2013
/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_2013.tif
2016


/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_2016.tif
2017


/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_2017.tif
2018
/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_2018.tif
2020


/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_2020.tif


In [ ]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##
# i had trouble getting correct land_use data using the script above, so i downloaded bridgets and tried that.
# the correct path should be the following if it worked
#/content/gdrive/MyDrive/Data/drainage_analysis/land_use/'+i+'_*.tif'
years = np.arange(1987,2021)
temporal_group = []
for i in names:
  files= sorted(glob.glob('/content/gdrive/MyDrive/Data/drainage_analysis/land_use/'+i+'_*.tif')) #name for all the csv files
  print(files)
  sta = SpatioTemporalAnalysis(files, dates=years, nodata=0) #import all CDL rasters and mask
  temporal_group.append(sta)


['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1996.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/West Hartley_1997.tif', '/content/gdrive/MyDrive/Da

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1996.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Sand Run Gulch_1997.tif', '/c

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1996.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Eagle Drain_1997.tif', '/content/gdrive/MyDrive/Data/drainage_

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1996.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Drain_1997.tif', '/content/gdrive/MyDrive/Data/drainage_

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1996.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Mason Creek_1997.tif', '/content/gdrive/MyDrive/Data/drainage_

...

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_1996.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/South Middleton_19

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1996.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Indian Creek_1997.tif', '/content/gdrive/MyDrive/Da

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 3_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Drainage District No. 

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Fifteen Mile Creek_1996.tif', '/content/gdrive/MyDrive/Data/drainage_ana

['/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1986.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1987.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1988.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1989.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1990.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1991.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1992.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1993.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1994.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1995.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1996.tif', '/content/gdrive/MyDrive/Data/drainage_analysis/land_use/Dixie drain_1997.tif', '/content/gdrive/MyDrive/Data/drainage_

In [ ]:
# ------------------------------- #
# CALCULATE THE CLASS PROPORTIONS #
# ------------------------------- #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('/content/gdrive/MyDrive/Data/drainage_analysis/land_use/proportions/'+names[i]+'_prop.csv')
  proportions.append(df)


In [ ]:
# ------------------------------- #
# CALCULATE CONFIGURATION METRICS #
# ------------------------------- #
#lots of problems with this code

config = []
!pip install psutil

for i in range(len(names)):
  print(f"Processing {names[i]}") #tells me what file it is struggling with
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['contagion','largest_patch_index']) #'contagion' is problem
  print(type(df))
  df.to_csv('/content/gdrive/MyDrive/Data/drainage_analysis/land_use/configuration/'+names[i]+'_configuration.csv')
  config.append(df)
  print(f"Finished {names[i]}") #tells me what file it is struggling with
  #memory = psutil.birtual_memory()
  #print(f"Total Memory: {memory.total} bytes")
  #print(f"Available Memory: {memory.available} bytes")
  #print(f"Used Memory: {memory.used} bytes")
  del(df)

Processing West Hartley
<class 'pandas.core.frame.DataFrame'>
Finished West Hartley
Processing Sand Run Gulch
<class 'pandas.core.frame.DataFrame'>
Finished Sand Run Gulch
Processing Eagle Drain
<class 'pandas.core.frame.DataFrame'>
Finished Eagle Drain
Processing Mason Drain
<class 'pandas.core.frame.DataFrame'>
Finished Mason Drain
Processing Mason Creek
<class 'pandas.core.frame.DataFrame'>
Finished Mason Creek
Processing East Hartley Drain
<class 'pandas.core.frame.DataFrame'>
Finished East Hartley Drain
Processing Willow Creek
<class 'pandas.core.frame.DataFrame'>
Finished Willow Creek
Processing Conway Gulch
<class 'pandas.core.frame.DataFrame'>
Finished Conway Gulch
Processing ThurmanDrain
<class 'pandas.core.frame.DataFrame'>
Finished ThurmanDrain
Processing North Middleton
<class 'pandas.core.frame.DataFrame'>
Finished North Middleton
Processing South Middleton
<class 'pandas.core.frame.DataFrame'>
Finished South Middleton
Processing Indian Creek
<class 'pandas.core.frame.Data

In [ ]:
# ----------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics #
# ----------------------------------------------------------------- #

# Import csv files into a list of dataframes

files = sorted(glob.glob('/content/gdrive/MyDrive/Data/drainage_analysis/land_use/proportions/*_prop.csv'))
files_config = sorted(glob.glob('/content/gdrive/MyDrive/Data/drainage_analysis/land_use/configuration/*_configuration.csv'))
names = list(sorted(shp_file['Name']))

proportions = []
for i in files:
  data = pd.read_csv(i)
  proportions.append(data)
config = []
for i in files_config:
  data = pd.read_csv(i)
  config.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DrainName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)


In [ ]:
## ------------------------------------------ ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING ##
## ------------------------------------------ ##

prop = pd.concat(new_df)

change = prop.groupby('DrainName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('/content/gdrive/MyDrive/Data/drainage_analysis/land_use/proportions/change.csv')


In [ ]:
## -------------------- ##
## MERGE TWO DATAFRAMES ##
## -------------------- ##

merged = []

for i in range(len(new_df)):
  df = new_df[i]
  con = config[i]
  df_merge = df.merge(con, on='dates', how='left')
  display(df_merge)
  df_merge.to_csv('/content/gdrive/MyDrive/Data/drainage_analysis/output_files/land_cover/'+names[i]+'.csv')
  merged.append(df_merge)

,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Conway Gulch,3.265995,32.229335,79.769058,92.974939
1,1988,Conway Gulch,3.273661,30.662782,83.679214,92.521072
2,1989,Conway Gulch,3.257050,30.572060,83.505170,92.380120
3,1990,Conway Gulch,3.265995,30.377838,83.412106,92.315282
4,1991,Conway Gulch,3.271106,30.306283,83.416387,92.250444
5,1992,Conway Gulch,3.337550,30.193839,83.177831,92.109492
6,1993,Conway Gulch,3.375883,30.059672,78.682895,92.022101
7,1994,Conway Gulch,3.433383,30.037950,78.699054,91.993911
8,1995,Conway Gulch,3.483216,29.782395,78.397877,91.852959
9,1996,Conway Gulch,3.520272,29.657173,82.802453,91.782482


,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Dixie drain,2.449842,55.659475,81.342650,86.564488
1,1988,Dixie drain,2.438658,53.403630,81.200713,86.326169
2,1989,Dixie drain,2.437500,53.201148,81.057875,86.156358
3,1990,Dixie drain,2.456013,52.579817,80.971270,85.994563
4,1991,Dixie drain,2.480697,52.326810,80.925648,85.909293
5,1992,Dixie drain,2.543177,52.251988,80.861987,85.764990
6,1993,Dixie drain,2.592158,52.169453,80.711606,85.598093
7,1994,Dixie drain,2.628798,52.154797,80.596380,85.426096
8,1995,Dixie drain,2.635740,52.017109,82.426472,85.219115
9,1996,Dixie drain,2.653867,52.162510,82.166778,84.953830


,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Drainage District No. 3,26.876726,17.604820,76.099575,86.032995
1,1988,Drainage District No. 3,28.086869,16.492594,80.299425,88.216991
2,1989,Drainage District No. 3,28.779814,15.777052,79.857796,90.130922
3,1990,Drainage District No. 3,29.834296,14.865679,82.209107,91.921564
4,1991,Drainage District No. 3,31.102184,13.788602,85.613034,93.929431
5,1992,Drainage District No. 3,32.287221,12.806929,88.435304,95.496976
6,1993,Drainage District No. 3,34.443887,10.803415,91.058910,96.800329
7,1994,Drainage District No. 3,36.472508,8.905348,93.275198,97.634005
8,1995,Drainage District No. 3,38.767261,6.781321,94.740344,98.162391
9,1996,Drainage District No. 3,40.311323,5.802159,95.111347,98.273939


,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Eagle Drain,2.430078,96.515360,63.035578,55.076575
1,1988,Eagle Drain,2.453003,96.469509,62.774000,54.923425
2,1989,Eagle Drain,2.475928,96.377808,62.823582,54.910871
3,1990,Eagle Drain,2.567630,96.286107,62.684639,54.652272
4,1991,Eagle Drain,2.636405,96.217331,63.257749,54.539292
5,1992,Eagle Drain,2.751032,96.102705,63.716523,54.373588
6,1993,Eagle Drain,2.773957,96.033929,64.731900,54.212905
7,1994,Eagle Drain,2.728106,96.056855,65.476604,53.474768
8,1995,Eagle Drain,3.003210,95.460798,66.862967,53.341702
9,1996,Eagle Drain,3.094911,95.071068,67.854786,52.887271


,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,East Hartley Drain,23.600817,75.411692,67.924385,76.296587
1,1988,East Hartley Drain,24.551903,74.466863,67.718537,75.828154
2,1989,East Hartley Drain,25.227106,73.843423,67.544271,75.714312
3,1990,East Hartley Drain,25.856802,73.213158,67.530055,75.520221
4,1991,East Hartley Drain,26.494462,72.608490,67.633185,75.482896
5,1992,East Hartley Drain,27.197538,71.919635,67.523038,75.383984
6,1993,East Hartley Drain,28.438728,70.723952,67.621602,75.305601
7,1994,East Hartley Drain,29.604264,69.560123,67.928555,75.303734
8,1995,East Hartley Drain,30.605976,68.562392,67.884384,75.219753
9,1996,East Hartley Drain,31.373330,67.789350,67.949489,75.156300


...

,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Sand Run Gulch,8.091295,57.117449,65.931489,46.619511
1,1988,Sand Run Gulch,8.309458,56.744206,67.790013,46.218403
2,1989,Sand Run Gulch,8.486441,56.548824,67.935456,46.135096
3,1990,Sand Run Gulch,8.627503,56.390240,71.692058,45.643739
4,1991,Sand Run Gulch,8.864941,56.151925,71.918851,45.425058
5,1992,Sand Run Gulch,9.240811,55.830376,74.115487,45.397675
6,1993,Sand Run Gulch,9.619310,55.492180,71.854053,45.345608
7,1994,Sand Run Gulch,9.944364,55.192535,71.726814,29.575520
8,1995,Sand Run Gulch,10.575196,54.516143,71.863252,29.514968
9,1996,Sand Run Gulch,10.997503,54.107855,68.482813,30.835538


,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,South Middleton,8.526864,21.146327,64.376763,56.031892
1,1988,South Middleton,8.781439,20.091398,64.346656,55.783064
2,1989,South Middleton,8.938834,19.508085,64.183073,55.602576
3,1990,South Middleton,9.195451,18.949833,64.046627,55.465019
4,1991,South Middleton,9.487334,18.677568,63.834565,55.040084
5,1992,South Middleton,9.876057,18.279999,63.516006,54.746572
6,1993,South Middleton,10.410156,17.720953,63.118758,54.374206
7,1994,South Middleton,10.872701,16.998730,62.779220,54.100845
8,1995,South Middleton,11.130225,16.457601,62.274221,52.726158
9,1996,South Middleton,11.107545,15.996190,61.950899,52.325755


,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,ThurmanDrain,86.532026,11.988493,73.908225,74.710040
1,1988,ThurmanDrain,88.575119,9.874949,66.927270,76.948170
2,1989,ThurmanDrain,90.248341,8.154758,67.417331,78.334542
3,1990,ThurmanDrain,92.044854,6.399342,68.503266,79.603117
4,1991,ThurmanDrain,94.040979,4.561733,71.584558,83.281986
5,1992,ThurmanDrain,95.561557,3.123349,74.369928,86.951794
6,1993,ThurmanDrain,96.859038,1.861093,80.949380,89.624864
7,1994,ThurmanDrain,97.686843,1.127224,82.855178,91.301196
8,1995,ThurmanDrain,98.215229,0.598838,85.813307,93.503081
9,1996,ThurmanDrain,98.326777,0.469677,87.748604,94.572309


,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,West Hartley,24.970941,23.592210,69.191049,63.755894
1,1988,West Hartley,25.565401,22.493738,71.338449,65.268780
2,1989,West Hartley,26.025041,21.867800,71.511245,65.344169
3,1990,West Hartley,26.625190,21.087701,71.728733,65.486002
4,1991,West Hartley,27.333613,20.574778,71.832560,65.544780
5,1992,West Hartley,28.096456,20.034738,71.805559,65.584391
6,1993,West Hartley,29.529988,19.094410,72.011821,65.698113
7,1994,West Hartley,30.996892,17.892406,71.674534,65.644446
8,1995,West Hartley,32.039426,16.960610,71.635381,65.842501
9,1996,West Hartley,32.807958,16.370511,71.590580,65.954946


,dates,DrainName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,Willow Creek,12.232984,87.431766,82.819880,83.808368
1,1988,Willow Creek,12.469117,87.166481,83.524880,84.210505
2,1989,Willow Creek,12.597387,87.027279,83.593073,84.275273
3,1990,Willow Creek,12.719097,86.896823,83.670073,84.358488
4,1991,Willow Creek,12.832062,86.789689,83.678485,84.381034
5,1992,Willow Creek,12.974907,86.660690,83.537666,84.347830
6,1993,Willow Creek,13.174600,86.447880,83.588425,84.377344
7,1994,Willow Creek,13.342225,86.270780,83.630523,84.437193
8,1995,Willow Creek,13.578357,86.025902,83.516954,84.409728
9,1996,Willow Creek,13.844371,85.697940,83.532176,84.394151
